In [1]:
import pandas as pd
import numpy as np


dfN = pd.read_csv('NaturalFires/NaturalFires.csv')
print(len(dfN))

idx = 0

print(dfN["hourly.time"].iloc[idx])

dfh = pd.read_csv(f'OUTPUTFWI/fw_{idx+1}.csv')

print(dfh["hourly.time"].iloc[-1])




1770
2001-07-20T15:00
2001-07-20 23:00:00


In [11]:
print(pd.to_datetime(dfN["hourly.time"].iloc[idx]).hour)

14


In [2]:
idx = 0
df_list = []
for nf in range(len(dfN)):
    dfhC = pd.read_csv(f'OUTPUTNOFIREFWI/fw_{idx}.csv')
    dfhC = dfhC.iloc[48:]
    dfhC = dfhC.tail(120)
    

    dfhC['hourly.time'] = pd.to_datetime(dfhC['hourly.time'])
    #dfhC.set_index('hourly.time', inplace=True)

    #df1_daily = dfhC.resample('D').mean()

    idx += 1

    dfhC["location"] = idx
    dfhC['wildfire'] = 0

    #df1_daily['wildfire'] = 0
    #df1_daily["location"] = idx+1
    #df1_daily.iloc[-1, df1_daily.columns.get_loc('wildfire')] = 1

    #df1_daily = df1_daily.tail(5)

    hora_incendio = int(pd.to_datetime(dfN["hourly.time"].iloc[idx]).hour)

    df1_last_24_hours = dfhC.iloc[-24:].loc[(dfhC['hourly.time'].dt.hour >= hora_incendio - 3) & (dfhC['hourly.time'].dt.hour < hora_incendio + 1)]

    dfhC.loc[df1_last_24_hours.index, 'wildfire'] = 1

    #dfhC.iloc[-24:, dfhC.columns.get_loc('wildfire')] = 1

    df_list.append(dfhC)
    
df = pd.concat(df_list)

EmptyDataError: No columns to parse from file

In [6]:
#hourly.time
df = df.drop(['index','latitude','hourly.is_day','longitude','elevation','hourly.snowfall', 'hourly.snow_depth', 'hourly.weather_code'], axis=1)


df_numeric = df.select_dtypes(include=[np.number])
correlations = df_numeric.corrwith(df_numeric['wildfire']).sort_values(ascending=False)

print(df.columns)

KeyError: "['index', 'latitude', 'hourly.is_day', 'longitude', 'elevation', 'hourly.snowfall', 'hourly.snow_depth', 'hourly.weather_code'] not found in axis"

In [5]:
from sklearn.ensemble import RandomForestClassifier

X = df_numeric.drop('wildfire', axis=1)
y = df_numeric['wildfire']

model = RandomForestClassifier(random_state=145521151)
model.fit(X, y)

importances = pd.DataFrame({'feature':X.columns,'importance':np.round(model.feature_importances_,3)})
importances = importances.sort_values('importance',ascending=False)

print("Correlations:\n", correlations)
print("\nFeature Importances:\n", importances)

Correlations:
 wildfire                                   1.000000e+00
hourly.vapour_pressure_deficit             5.024468e-01
hourly.apparent_temperature                4.098440e-01
hourly.temperature_2m                      4.068996e-01
hourly.et0_fao_evapotranspiration          3.576700e-01
hourly.soil_temperature_0_to_7cm           3.545200e-01
ffmc                                       3.543412e-01
hourly.direct_radiation                    3.139226e-01
hourly.direct_radiation_instant            3.086948e-01
hourly.global_tilted_irradiance            2.990074e-01
hourly.shortwave_radiation                 2.990074e-01
hourly.shortwave_radiation_instant         2.928431e-01
hourly.global_tilted_irradiance_instant    2.927456e-01
hourly.terrestrial_radiation               2.832019e-01
hourly.soil_temperature_100_to_255cm       2.798054e-01
hourly.terrestrial_radiation_instant       2.760989e-01
isi                                        2.625658e-01
fwix                             